In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import random
import unicodedata
import re
from selenium.webdriver.common.by import By

In [2]:
#url파일 가져오기
df = pd.read_excel("/Users/son-yejin/데이트폭력_네이버기사_url.xlsx")

In [3]:
# df의 https://n.news.naver.com/mnews/article 으로 시작하는 것만 가져오기
need = []
string = "https://n.news.naver.com/mnews/article"
for i in range(len(df)):
    if string in df['url'].iloc[i]:
        need.append(i)
df_ = df['url'].loc[need]
df_= pd.DataFrame(df_)
df_.reset_index(inplace = True)

In [4]:
# 빈 데이터프레임, 행의 크기는 url 데이터프레임의 크기정도로 잡습니다.
DF = pd.DataFrame(columns = ['헤드라인','날짜', '언론사','리포터', '본문', '댓글', 'url'],index = list(range(0,40000)))

In [5]:
headline = '//*[@id="ct"]/div[1]/div[2]/h2'
reporter = '//*[@id="ct"]/div[1]/div[3]/div[2]/a/em'
date = '//*[@id="ct"]/div[1]/div[3]/div[1]/div/span'
content = '//*[@id="dic_area"]'
press = '//*[@id="ct"]/div[1]/div[1]/a/img[1]'

In [6]:
# 본인의 드라이버 경로 입력하기
driver = webdriver.Chrome('/Users/son-yejin/Downloads/chromedriver 4')

/var/folders/pb/ct7cxfvs6477s2t94xjdnrnm0000gn/T/ipykernel_79602/292541899.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/son-yejin/Downloads/chromedriver 4')


In [7]:
driver = webdriver.Chrome('/Users/son-yejin/Downloads/chromedriver 4')

num = 0
for i in range(len(df_)): 
    
    url = df_['url'].loc[i]   
    
    driver.get(url)
    driver.implicitly_wait(10)
    
    # 댓글 개수 체크 / 만약 "네이버 뉴스"가 아니라 "TV연예"에 떴을 경우 pass
    
    try: 
        
        댓글상태 = driver.find_element(By.XPATH,'//*[@id="cbox_module"]/div[2]/div[1]/a/span[1]')
        댓글상태 = 댓글상태.text
        
        if 댓글상태 == "댓글":
            continue
        elif int(댓글상태) != 0:

            헤드라인 = driver.find_element(By.XPATH, headline).text
            리포터 = driver.find_element(By.XPATH, reporter).text
            날짜 = driver.find_element(By.XPATH, date).text          
            내용 = driver.find_element(By.XPATH, content).text
            언론사 =  driver.find_element(By.XPATH, press).get_attribute("alt")

            try: 
                # 첫번째 댓글더보기
                댓글더보기 = '//*[@id="cbox_module"]/div[2]/div[9]/a/span[1]'
                driver.find_element(By.XPATH, 댓글더보기).click()
                # 두번째 이상의 댓글 더보기
                while True:
                    try:
                        더보기 = '//*[@id="cbox_module"]/div[2]/div[9]/a'
                        driver.find_element(By.XPATH, 더보기).click()
                    except:
                        break
            except:
                print("더보기없음")
                
            댓글 = driver.find_element(By.XPATH,'//*[@id="cbox_module_wai_u_cbox_content_wrap_tabpanel"]').text
            DF.iloc[num] = 헤드라인, 날짜,  언론사, 리포터,  내용, 댓글, url 
            num +=1
    except:
        pass


/var/folders/pb/ct7cxfvs6477s2t94xjdnrnm0000gn/T/ipykernel_79602/3656090647.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/son-yejin/Downloads/chromedriver 4')


In [8]:
DF.head()

,헤드라인,날짜,언론사,리포터,본문,댓글,url
0,"'데이트 폭력'에 덜미 지명수배범, 파출소에서 도주",2022.07.27. 오전 9:49,YTN,나현호 기자,이른바 '데이트 폭력'을 저지르다 붙잡힌 남성이 파출소에서 달아나 경찰이 추적에 나...,hg02****\n댓글모음\n옵션 열기\n수갑 채우지 말라는거 인권위 권고사항 아니...,https://n.news.naver.com/mnews/article/052/000...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
DF.to_csv("데이트폭력.csv")